In [ ]:
!pip install transformers
!pip install sentencepiece
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
import shutil
import time
import gc
import random
import math
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
os.environ["WANDB_DISABLED"] = "true"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Config

In [ ]:
class CFG:
    input_path = '/content/drive/MyDrive/us-patent/'
#     model_path = 'microsoft/deberta-v3-small' #  nghuyong/ernie-2.0-large-en studio-ousia/luke-large
    # model_path = '/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large'
    model_path = '/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large'
    # model_path = '/content/drive/MyDrive/us-patent/deberta-v3-small' 
    # model_path_save = '/content/drive/MyDrive/us-patent/smallmodel/',
    model_path_save = '/content/drive/MyDrive/us-patent/tmp/large4/',   

    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5  # 1.5
    num_warmup_steps = 0.1
    max_input_length = 140
    epochs = 4  # 5
    min_lr = 0.5e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 4
    batch_size = 8#100
    seed = 86#19800102
    OUTPUT_DIR = './train_checkpoint'
    num_workers = 2#2
    device='cuda'
    print_freq = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## logger

In [ ]:
def get_logger(filename=CFG.OUTPUT_DIR+ 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()
LOGGER.info('===============lr_{}==============='.format(CFG.learning_rate))
LOGGER.info('===============seed_{}==============='.format(CFG.seed))
LOGGER.info('===============total_epochs_{}==============='.format(CFG.epochs))
LOGGER.info('===============num_warmup_steps_{}==============='.format(CFG.num_warmup_steps))

===============lr_2e-05===============
===============lr_2e-05===============
===============seed_86===============
===============seed_86===============
===============total_epochs_4===============
===============total_epochs_4===============
===============num_warmup_steps_0.1===============
===============num_warmup_steps_0.1===============


In [ ]:
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d helloggfss/foldsdump
# !unzip /content/foldsdump.zip -d /content/drive/MyDrive/us-patent/folds-dump-the-two-paths-fix


# Preproc

In [ ]:
# improve1
# ====================================================
# CV split
# ====================================================
train_df = pd.read_csv("/content/drive/MyDrive/us-patent/us-patent-phrase-to-phrase-matching/train.csv")
cpc_texts = torch.load("/content/drive/MyDrive/us-patent/folds-dump-the-two-paths-fix/cpc_texts_fixed.pth")
train_df['context_text'] = train_df['context'].map(cpc_texts)

train_df['score_map'] = train_df['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
Fold = StratifiedKFold(n_splits=CFG.num_fold, shuffle=True, random_state=CFG.seed)
# for n, (train_index, val_index) in enumerate(Fold.split(train_df, train_df['score_map'])):
for n, (train_index, val_index) in enumerate(Fold.split(train_df, train_df['anchor'])):
    train_df.loc[val_index, 'fold'] = int(n)
train_df['fold'] = train_df['fold'].astype(int)
display(train_df.groupby('fold').size())
train_df['input'] = train_df['anchor'] + ' ' + train_df['context_text'].apply(str.lower) + ' ' + train_df['target'].apply(str.lower)


# train_df = pd.read_csv("/content/drive/MyDrive/us-patent/folds-dump-the-two-paths-fix/train_folds_5.csv")
# train_df['input'] = train_df['anchor'] + ' ' + train_df['context_text_fix'].apply(str.lower) + ' ' + train_df['target'].apply(str.lower)
# train_df

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  UserWarning,


fold
0    9119
1    9118
2    9118
3    9118
dtype: int64

In [ ]:
train_df

,id,anchor,target,context,score,context_text,score_map,fold,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,2,0,abatement human necessities. furniture; domest...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,3,3,abatement human necessities. furniture; domest...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,1,2,abatement human necessities. furniture; domest...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,2,0,abatement human necessities. furniture; domest...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,0,abatement human necessities. furniture; domest...
...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,4,1,wood article performing operations; transporti...
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,2,1,wood article performing operations; transporti...
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,2,2,wood article performing operations; transporti...
36471,756ec035e694722b,wood article,wooden material,B44,0.75,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,3,1,wood article performing operations; transporti...


# Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

loading configuration file /content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.19.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

Didn't find file /content/drive/MyDr

# Dataset

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.texts = df['input'].values.astype(str)
        
        self.label = df['score'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.label[item]
        
        
        inputs = tokenizer(text,
                    max_length=CFG.max_input_length,
                    padding='max_length',
                    truncation=True )
        return {**inputs,
               'labels':torch.as_tensor(label, dtype=torch.float) }


## Model

In [ ]:
class Custom_Bert_Simple(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(CFG.model_path)
        self.base = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, token_type_ids ,attention_mask, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids
                               )

        output = base_output[0]
        if labels is None:
            return output
        
        else:
            return (nn.MSELoss()(torch.squeeze(output,1),labels), output)

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
model = Custom_Bert_Simple()

loading configuration file /content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.19.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

loading configuration file /content/

## Trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # forward pass
        outputs = model(**inputs)
        loss, logits = outputs
        return (loss, outputs) if return_outputs else loss

# Train

In [ ]:
def compute_metrics(eval_pred):
    torch.cuda.empty_cache()
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    torch.cuda.empty_cache()
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(CFG.seed)

In [ ]:
# improve1
# ====================================================
# Define max_len
# ====================================================
from tqdm.auto import tqdm

lengths_dict = {}

lengths = []
tk0 = tqdm(cpc_texts.values(), total=len(cpc_texts))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
lengths_dict['context_text'] = lengths

for text_col in ['anchor', 'target']:
    lengths = []
    tk0 = tqdm(train_df[text_col].fillna("").values, total=len(train_df))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths

oof_df = pd.DataFrame()
for fold in range(CFG.num_fold):
    
    #improve1
    CFG.max_input_length = max(lengths_dict['anchor']) + max(lengths_dict['target'])\
                + max(lengths_dict['context_text']) + 4 # CLS + SEP + SEP + SEP
    print(f"max_input_length: {CFG.max_input_length}")    

    tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
    va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
    tr_dataset = TrainDataset(tr_data)
    va_dataset = TrainDataset(va_data)
    args = TrainingArguments(
        CFG.OUTPUT_DIR,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=CFG.learning_rate,
        per_device_train_batch_size=CFG.batch_size,
        per_device_eval_batch_size=CFG.batch_size,
        num_train_epochs=CFG.epochs,
        weight_decay=CFG.weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="pearson",
        save_total_limit=1,
        warmup_ratio=CFG.num_warmup_steps
    )
    model = Custom_Bert_Simple()
    torch.cuda.empty_cache()
    
    trainer = CustomTrainer(
        model,
        args,
        train_dataset=tr_dataset,
        eval_dataset=va_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    torch.cuda.empty_cache()    
    trainer.train()
    torch.cuda.empty_cache()

    outputs = trainer.predict(va_dataset)
    predictions = outputs.predictions.reshape(-1)
    va_data['preds'] = predictions
    torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        # '/content/drive/MyDrive/us-patent/smallmodel/' + "{}_best{}.pth".format(CFG.model_path.replace('/', '_'),fold))
                       '/content/drive/MyDrive/us-patent/tmp/large4/' + "{}_best{}.pth".format(CFG.model_path.replace('/', '_'),fold))

    shutil.rmtree(CFG.OUTPUT_DIR)
    oof_df = pd.concat([oof_df, va_data])

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/36473 [00:00<?, ?it/s]

  0%|          | 0/36473 [00:00<?, ?it/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file /content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type

max_input_length: 133


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file /content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "/content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type

max_input_length: 133


Some weights of the model checkpoint at /content/drive/MyDrive/us-patent/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifer.weight', 'mask_predictions.classifer.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Ber

Epoch,Training Loss,Validation Loss,Pearson
1,0.027900,0.029150,0.807015
2,0.017100,0.020945,0.836199
3,0.011400,0.019106,0.852812
4,0.006400,0.018391,0.857934


***** Running Evaluation *****
  Num examples = 9118
  Batch size = 8
Saving model checkpoint to ./train_checkpoint/checkpoint-3420
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./train_checkpoint/checkpoint-3420/tokenizer_config.json
Special tokens file saved in ./train_checkpoint/checkpoint-3420/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9118
  Batch size = 8
Saving model checkpoint to ./train_checkpoint/checkpoint-6840
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./train_checkpoint/checkpoint-6840/tokenizer_config.json
Special tokens file saved in ./train_checkpoint/checkpoint-6840/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9118
  Batch size = 8
Saving model checkpoint to ./train_checkpoint/checkpoint-10260
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./train_ch

In [ ]:
predictions = oof_df['preds'].values
label = oof_df['score'].values
eval_pred = predictions, label
compute_metrics(eval_pred)

{'pearson': 0.8621580950996016}

In [ ]:
oof_df.to_csv('/content/drive/MyDrive/us-patent/tmp/large4/oof_df.csv')
oof_df.to_pickle('/content/drive/MyDrive/us-patent/tmp/large4/oof_df.pkl')